In [6]:
import sqlite3

# Chemin vers votre fichier de base de données SQLite
database_path = 'puissance4.db'

# Connexion à la base de données
conn = sqlite3.connect(database_path)

# Création d'un curseur pour exécuter des requêtes SQL
cur = conn.cursor()

try:
    # Requête SQL pour sélectionner toutes les lignes de la table game_results
    query = "SELECT * FROM game_results"
    
    # Exécution de la requête
    cur.execute(query)
    
    # Récupération de tous les résultats
    results = cur.fetchall()

    # Affichage des résultats avec game_mode
    print("ID | Winner | Move Count | Loser | Game Mode")
    print("--------------------------------------------")
    for row in results:
        # Adaptation en fonction de la longueur de la ligne pour éviter des erreurs si une colonne attendue n'existe pas
        print(f"{row[0]} | {row[1]} | {row[2]} | {row[3]} | {row[4]}")
        
except sqlite3.Error as e:
    print(f"Une erreur est survenue lors de la lecture de la base de données: {e}")

# Fermeture de la connexion
cur.close()
conn.close()


ID | Winner | Move Count | Loser | Game Mode
--------------------------------------------
1 | Joueur 2 | 8 | Joueur 1 | Joueur vs Joueur
2 | Joueur 1 | 7 | Joueur 2 | Joueur vs Joueur
3 | Joueur 1 | 7 | Joueur 2 | Joueur vs Joueur
4 | Joueur 1 | 4 | Joueur Rouge | Joueur vs IA
5 | Joueur 1 | 7 | Joueur 2 | Joueur vs Joueur
6 | Joueur 1 | 7 | Joueur 2 | Joueur vs Joueur
7 | Joueur Rouge | 19 | IA Jaune | Joueur vs IA
8 | IA Jaune | 16 | Joueur Rouge | Joueur vs IA
9 | Joueur 1 | 7 | Joueur 2 | Joueur vs Joueur
10 | IA Jaune | 22 | Joueur Rouge | Joueur vs IA
11 | IA Jaune | 14 | Joueur Rouge | Joueur vs IA
12 | IA Jaune | 22 | Joueur Rouge | Joueur vs IA
13 | Joueur Rouge | 21 | IA Jaune | Joueur vs IA
14 | IA Jaune | 16 | Joueur Rouge | Joueur vs IA
15 | IA Jaune | 20 | Joueur Rouge | Joueur vs IA
16 | IA Jaune | 34 | Joueur Rouge | Joueur vs IA
17 | Joueur 1 | 41 | Joueur 2 | Joueur vs Joueur
18 | IA Jaune | 14 | Joueur Rouge | Joueur vs IA
19 | IA Jaune | 20 | Joueur Rouge | Joueur v

In [2]:
import dash
from dash import dcc, html, dash_table, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import sqlite3
import plotly.express as px
from dash.exceptions import PreventUpdate

# Chemin vers votre fichier de base de données SQLite
database_path = 'puissance4.db'

# Connexion à la base de données et récupération des données
conn = sqlite3.connect(database_path)
query = "SELECT * FROM game_results"
data = pd.read_sql_query(query, conn)
conn.close()

# Préparation des données pour l'analyse et la visualisation
data['winner'] = data['winner'].astype(str)
data['loser'] = data['loser'].astype(str)
data['move_count'] = data['move_count'].astype(int)

data['winner'] = data['winner'].apply(lambda x: x + ' 🟢' if x != 'None' else x)
data['loser'] = data['loser'].apply(lambda x: x + ' 🔴' if x != 'None' else x)

# Initialisation de l'application Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SOLAR])

# Définition du layout de l'application
app.layout = dbc.Container([
    html.H1("Analyse des Performances - Jeu Puissance 4", className="mb-5 text-primary"),
    html.H2("Tableau des Résultats :", className="mb-4 text-secondary"),
    dash_table.DataTable(
        id='results-table',
        columns=[{"name": i, "id": i} for i in data.columns],
        data=data.to_dict('records'),
        style_cell={'textAlign': 'left', 'color': 'black'},
        style_header={
            'backgroundColor': 'rgb(10, 10, 10)',
            'color': 'white'
        },
        style_data={
            'backgroundColor': 'rgb(200, 200, 200)',
            'color': 'black'
        },
    ),
    dbc.Row([
        dbc.Col(dbc.Button("Joueur 1", id="btn-joueur1", color="primary"), width="auto"),
        dbc.Col(dbc.Button("Joueur 2", id="btn-joueur2", color="success"), width="auto"),
        dbc.Col(dbc.Button("IA", id="btn-ia", color="warning"), width="auto"),
    ], justify="center", className="mb-5"),
    dcc.Graph(id='performance-graph'),
    html.Div(id='stats-container'),
    dcc.Graph(id='move-count-graph'),
    html.Div(id='min-moves-win'),
    html.Div(id='win-probability', className="mb-4 text-info")  # Element pour afficher la probabilité de victoire
], fluid=True)

# Fonction pour calculer la probabilité de victoire
def calculate_win_probability(player_name):
    if player_name not in ['Joueur 1', 'Joueur 2', 'IA']:
        return 0  # Retourne une probabilité de 0 si le nom du joueur n'est pas reconnu
    
    total_games = len(data[(data['winner'].str.contains(player_name)) | (data['loser'].str.contains(player_name))])
    if total_games == 0:
        return 0  # Évite la division par zéro si le joueur n'a joué aucune partie
    
    wins = len(data[data['winner'].str.contains(player_name)])
    return wins / total_games

# Callback pour les boutons, mise à jour du graphique, des statistiques, et du compte de mouvements
@app.callback(
    [Output('performance-graph', 'figure'),
     Output('stats-container', 'children'),
     Output('move-count-graph', 'figure'),
     Output('min-moves-win', 'children'),
     Output('win-probability', 'children')],  # Output pour la probabilité de victoire
    [Input('btn-joueur1', 'n_clicks'),
     Input('btn-joueur2', 'n_clicks'),
     Input('btn-ia', 'n_clicks')]
)
def update_graph_and_moves(btn_joueur1, btn_joueur2, btn_ia):
    ctx = dash.callback_context

    if not ctx.triggered:
        # Si aucun bouton n'a été cliqué, ne faites rien.
        raise PreventUpdate

    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    player_name = None

    if button_id == "btn-joueur1" and btn_joueur1:
        player_name = "Joueur 1"
    elif button_id == "btn-joueur2" and btn_joueur2:
        player_name = "Joueur 2"
    elif button_id == "btn-ia" and btn_ia:
        player_name = "IA"

    if player_name is None:
        raise PreventUpdate

    filtered_data_winner = data[data['winner'].str.contains(player_name)]
    filtered_data_loser = data[data['loser'].str.contains(player_name)]

    victories_count = len(filtered_data_winner)
    losses_count = len(filtered_data_loser)

    stats = f"{player_name} - Victoires: {victories_count}, Défaites: {losses_count}"

    fig = px.bar(x=["Victoires", "Défaites"], y=[victories_count, losses_count], title=f"Performances de {player_name}", labels={'x': "Résultat", 'y': "Nombre"})

    if not data.empty and 'move_count' in data.columns and data['move_count'].min() > 0:
        min_moves = data[data['move_count'] == data['move_count'].min()]
        min_moves_text = f"Le jeu gagné avec le moins de mouvements a été réalisé en {min_moves['move_count'].iloc[0]} mouvements par {min_moves['winner'].iloc[0]}."
    else:
        min_moves_text = "Aucune donnée disponible sur les mouvements."

    fig_moves = px.histogram(data, x='move_count', title="Distribution du Nombre de Mouvements pour Gagner", labels={'move_count': 'Nombre de Mouvements'})

    # Calcul de la probabilité de victoire
    win_probability = calculate_win_probability(player_name)
    win_probability_text = f"Probabilité de victoire pour {player_name} : {win_probability:.2%}"

    return fig, stats, fig_moves, min_moves_text, win_probability_text

if __name__ == '__main__':
    app.run_server(debug=True)
